In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats

from gm import gm
from gm_Prep import gm_Prep

In [ ]:
# np.random.seed(1337)

In [ ]:
# SBML model we want to import
sbml_file = 'BigModel_FULL_v4.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4] # 'BigModel_byparts_v1'
# Directory to which the generated model code is written
model_output_dir = model_name

kGsRead = pd.read_excel('kGeneMod.xlsx',header=0,index_col=0)
gExp_mpc = np.float64(kGsRead.values[:,0])
mExp_mpc = np.float64(kGsRead.values[:,1])
kGin = np.float64(kGsRead.values[:,2])
kGac = np.float64(kGsRead.values[:,3])
kTCleak = np.float64(kGsRead.values[:,4])
kTCmaxs = np.float64(kGsRead.values[:,5])
kTCd = np.float64(kGsRead.values[:,6])
kmRNAs = kGsRead.values[:,7]

# Read-in the activators matrix and assign concentrations of activators
TAsRead = pd.read_csv('TAs.csv',header=0,index_col=0)
TAs0 = np.float64(TAsRead.values)
# Read-in the repressors matrix and assign concentrations of repressors
TRsRead = pd.read_csv('TRs.csv',header=0,index_col=0)
TRs0 = np.float64(TRsRead.values)

In [ ]:
flagD = 0
th = 48
numStocCells = 25
STIMligs = [3.3,0.0,0.0,0.0,0.0,0.0,1721.0]
nmxlsfile1 = 'Fig5C_BAD_25cells_48hrs_'
nmxlsfile2 = 'Fig5C_BIM_25cells_48hrs_'

ts = 30
NSteps = th*3600/ts
NSteps = int(NSteps)
Vn = 1.75E-12 # get from model somehow
Vc = 5.25E-12 # get from model somehow
tout_all = np.arange(0,th*3600+1,30)

startTime = datetime.now()
print(startTime)

genedata0, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)
sys.path.insert(0, os.path.abspath(model_output_dir))
currentDirectory = os.getcwd()

namexlsS = 'NoGF_100cells_24of72hrs_S.xlsx'
condsS0 = pd.read_excel(namexlsS,header=0,index_col=0)
spdataMat0 = condsS0.loc[:,1:]
namexlsG = 'NoGF_100cells_24of72hrs_G.xlsx'
condsG0 = pd.read_excel(namexlsG,header=0,index_col=0)
genedataMat0 = condsG0.loc[:,1:]

for nn in range(numStocCells):
    spdata = np.array(spdataMat0.loc[nn,:])
    spdata[155:162] = STIMligs
    xoutS_all = np.zeros(shape=(NSteps+1,len(spdata)))
    xoutS_all[0,:] = spdata # 24hr time point     
    genedata = np.array(genedataMat0.loc[nn,:])
    xoutG_all = np.zeros(shape=(NSteps+1,len(genedata)))
    xoutG_all[0,:] = genedata
    model_module = importlib.import_module(model_name)
    model = model_module.getModel()
    solver = model.getSolver() # Create solver instance
    solver.setMaxSteps = 1e10
    model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints
    model.setFixedParameterById('k1837',0.0)
    model.setFixedParameterById('k1840',0.0)
    for qq in range(NSteps):
        genedata,AllGenesVec = gm(flagD,ts,xoutG_all[qq,:],xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                   kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
        for ii,kk in enumerate(kmRNAs):
            model.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
        model.setInitialStates(xoutS_all[qq,:])
        rdata = amici.runAmiciSimulation(model, solver)  # Run simulation
        xoutS_all[qq+1,:] = rdata['x'][-1,:]
        xoutG_all[qq+1,:] = genedata
        if rdata['x'][-1,103] < rdata['x'][-1,105]:
            print('Apoptosis happened')
            break
    xoutS_all = xoutS_all[~np.all(xoutS_all == 0, axis=1)]
    xoutG_all = xoutG_all[~np.all(xoutG_all == 0, axis=1)]
    condsSDF = pd.DataFrame(data=xoutS_all,columns=[ele for ele in model.getStateIds()]) 
    condsSDF.to_excel(nmxlsfile1+'S_'+str(nn)+'.xlsx')    
    condsSDF = None 
print(datetime.now())